In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from pprint import pprint
from functools import lru_cache
import numpy as np
from enum import Enum, IntEnum
from string import ascii_lowercase, ascii_uppercase

In [3]:
DAY = 10

In [4]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'noop\naddx 22\naddx -17\naddx 1\naddx 4\naddx 17\naddx -16\naddx 4\naddx 1\naddx 21\naddx '
Split Data:
['noop',
 'addx 22',
 'addx -17',
 'addx 1',
 'addx 4',
 'addx 17',
 'addx -16',
 'addx 4',
 'addx 1',
 'addx 21']


In [75]:
test_data = ["addx 15",
"addx -11",
"addx 6",
"addx -3",
"addx 5",
"addx -1",
"addx -8",
"addx 13",
"addx 4",
"noop",
"addx -1",
"addx 5",
"addx -1",
"addx 5",
"addx -1",
"addx 5",
"addx -1",
"addx 5",
"addx -1",
"addx -35",
"addx 1",
"addx 24",
"addx -19",
"addx 1",
"addx 16",
"addx -11",
"noop",
"noop",
"addx 21",
"addx -15",
"noop",
"noop",
"addx -3",
"addx 9",
"addx 1",
"addx -3",
"addx 8",
"addx 1",
"addx 5",
"noop",
"noop",
"noop",
"noop",
"noop",
"addx -36",
"noop",
"addx 1",
"addx 7",
"noop",
"noop",
"noop",
"addx 2",
"addx 6",
"noop",
"noop",
"noop",
"noop",
"noop",
"addx 1",
"noop",
"noop",
"addx 7",
"addx 1",
"noop",
"addx -13",
"addx 13",
"addx 7",
"noop",
"addx 1",
"addx -33",
"noop",
"noop",
"noop",
"addx 2",
"noop",
"noop",
"noop",
"addx 8",
"noop",
"addx -1",
"addx 2",
"addx 1",
"noop",
"addx 17",
"addx -9",
"addx 1",
"addx 1",
"addx -3",
"addx 11",
"noop",
"noop",
"addx 1",
"noop",
"addx 1",
"noop",
"noop",
"addx -13",
"addx -19",
"addx 1",
"addx 3",
"addx 26",
"addx -30",
"addx 12",
"addx -1",
"addx 3",
"addx 1",
"noop",
"noop",
"noop",
"addx -9",
"addx 18",
"addx 1",
"addx 2",
"noop",
"noop",
"addx 9",
"noop",
"noop",
"noop",
"addx -1",
"addx 2",
"addx -37",
"addx 1",
"addx 3",
"noop",
"addx 15",
"addx -21",
"addx 22",
"addx -6",
"addx 1",
"noop",
"addx 2",
"addx 1",
"noop",
"addx -10",
"noop",
"noop",
"addx 20",
"addx 1",
"addx 2",
"addx 2",
"addx -6",
"addx -11",
"noop",
"noop",
"noop"]

In [39]:
test_data = [
"noop",
"addx 3",
"addx -5",
]

In [40]:
line_re = re.compile(r"^addx\s+(-?)(\d+)$")

In [41]:
def parse_line(line):
    if line == "noop":
        return (0, 1)
    elif line.startswith("addx"):
        negative, number = line_re.match(line).groups()
        number = int(number)
        if negative:
            number = -number
        return (number, 2)
    else:
        raise ValueError

In [87]:
def target_generator():
    t = 20
    yield t
    while True:
        if t > 220:
            raise StopIteration
        t += 40
        yield t

In [91]:
def run_program(str_data, targets):
    x=1
    cycles=1
    strength = 0
    # print(f"{'x':<3} {'cyc':<3} {'line':<10}  {'num':<3}")
    t = next(targets)
    for i, line in enumerate(str_data, start=1):
        number, cy_count = parse_line(line)
        print(f"{x:<3} {cycles:<3} {line:<10} {' ' if number>=0 else ''}{number:<3}")
        for _ in range(cy_count):
            if cycles == t:
                this_strength = (x * cycles)
                strength += this_strength
                t = next(targets)
                print("***",cycles, x, this_strength, strength)
            cycles += 1
            
        x += number
            
    return strength

In [93]:
first = run_program(data, target_generator())

1   1   noop        0  
1   2   addx 22     22 
23  4   addx -17   -17
6   6   addx 1      1  
7   8   addx 4      4  
11  10  addx 17     17 
28  12  addx -16   -16
12  14  addx 4      4  
16  16  addx 1      1  
17  18  addx 21     21 
38  20  addx -17   -17
*** 20 38 760 760
21  22  addx -10   -10
11  24  noop        0  
11  25  addx 17     17 
28  27  addx -1    -1 
27  29  addx 5      5  
32  31  addx -1    -1 
31  33  noop        0  
31  34  addx 4      4  
35  36  addx 1      1  
36  38  noop        0  
36  39  addx -37   -37
-1  41  addx 5      5  
4   43  addx 27     27 
31  45  addx -22   -22
9   47  addx -2    -2 
7   49  addx 2      2  
9   51  addx 5      5  
14  53  addx 2      2  
16  55  addx 5      5  
21  57  noop        0  
21  58  noop        0  
21  59  addx -2    -2 
*** 60 21 1260 2020
19  61  addx 5      5  
24  63  addx 16     16 
40  65  addx -11   -11
29  67  addx -2    -2 
27  69  addx 2      2  
29  71  addx 5      5  
34  73  addx 2      2  
36  75  addx -

In [94]:
submit_answer(DAY, 1, first)

13860
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit. [Continue to Part Two]')


True

In [217]:
def cycle_to_screen_index(cycle, width):
    i = cycle - 1
    return (i // width, i)

In [218]:
def sprite_indices(x):
    return (x-1,x,x+1)

In [219]:
def print_screen(screen, sprite_indices, crt_index, width):
    for i,s in enumerate(screen):
        if i == crt_index:
            print("\x1b[45m"+s+"\x1b[0m",end="")
        elif i in sprite_indices:
            print("\x1b[44m"+s+"\x1b[0m",end="")
        else:
            print(s,end="")
        if ((i+1) % width) == 0:
            print()
    print()

In [224]:
def run_program_screen(str_data, targets, screen_size):
    WIDTH = 40
    HEIGHT = 6
    screen=['.'] * (WIDTH * HEIGHT)
    x=1
    cycles=1
    row_index, crt_index = cycle_to_screen_index(cycles, WIDTH)
    sprite = sprite_indices(x)
    # print_screen(screen, sprite, crt_index, WIDTH)
    strength = 0
    t = next(targets)
    for i, line in enumerate(str_data, start=1):
        number, cy_count = parse_line(line)
        # print(f"{x:<3} {cycles:<3} {line:<10} {' ' if number>=0 else ''}{number:<3}")
        for _ in range(cy_count):
            
            row_index, crt_index = cycle_to_screen_index(cycles, WIDTH)
            sprite = sprite_indices(x + (row_index * WIDTH))
            # print(cycles, x) #, crt_index, sprite)
            if crt_index in sprite:
                screen[crt_index] = "#"
            if cycles == t:
                this_strength = (x * cycles)
                strength += this_strength
                t = next(targets)
                # print("***",cycles, x, this_strength, strength)
            cycles += 1
            # print_screen(screen, sprite, crt_index, WIDTH)
            
        x += number
            
    print_screen(screen, sprite, crt_index, WIDTH)

In [226]:
run_program_screen(data, target_generator(), 240)

###..####.#..#.####..##....##..##..###..
#..#....#.#..#.#....#..#....#.#..#.#..#.
#..#...#..####.###..#.......#.#....###..
###...#...#..#.#....#.##....#.#....#..#.
#.#..#....#..#.#....#..#.#..#.#..#.#..#.
#..#.####.#..#.#.....###..##...##..###..



In [227]:
second = "RZHFGJCB"

In [228]:
submit_answer(DAY, 2, second)

'RZHFGJCB'
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit.You have completed Day 10! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True